In [ ]:
import json
import math
import os
import requests
import scipy
import tifffile

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

from datetime import datetime as dt
from scipy import stats as st
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import activations
from tensorflow.keras.layers import concatenate
from keras import metrics
from keras.models import Sequential
from keras.layers import Activation, Dense
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.ensemble import RandomForestClassifier

from keras.models import Sequential
from keras.layers import Dense

from google.colab import drive
from google.colab import files
drive.mount('/content/drive/')

Mounted at /content/drive/


**Read Tiles Dataset**

In [ ]:
tiles = pd.read_csv(r'/content/drive/MyDrive/capstone/Tiles_binned.csv')

In [ ]:
tiles.head(5)

,Tile_ID,Long2,Lat2,Long1,Lat1,Mid_lat,Mid_long,Stop_Signs,Paving_historical,Paving_future,Bus_stop,Collisions_Future,Collisions_Historical,RTTYP,bin
0,36,-122.514446,37.779636,-122.513306,37.778732,37.779184,-122.513876,0.0,0.0,0.0,0.0,0.0,0.0,M,A
1,37,-122.514446,37.778732,-122.513306,37.777829,37.778280,-122.513876,0.0,0.0,0.0,0.0,0.0,0.0,M,A
2,151,-122.513306,37.779636,-122.512166,37.778732,37.779184,-122.512736,0.0,0.0,0.0,0.0,0.0,0.0,M,A
3,152,-122.513306,37.778732,-122.512166,37.777829,37.778280,-122.512736,0.0,0.0,0.0,0.0,0.0,0.0,M,A
4,153,-122.513306,37.777829,-122.512166,37.776925,37.777377,-122.512736,0.0,0.0,0.0,0.0,0.0,0.0,M,A


In [ ]:
tiles = tiles.rename(columns={'bin': 'label'})

In [ ]:
tiles.columns

Index(['Tile_ID', 'Long2', 'Lat2', 'Long1', 'Lat1', 'Mid_lat', 'Mid_long',
       'Stop_Signs', 'Paving_historical', 'Paving_future', 'Bus_stop',
       'Collisions_Future', 'Collisions_Historical', 'RTTYP', 'label'],
      dtype='object')

In [ ]:
# x: 'Tile_ID', 'Mid_lat', 'Mid_long',
#    'Stop_Signs', 'Bus_stop', 'Paving', 'Future', 'Historical',
#    'Collision_binary_historical'
# y: 'Collision_binary_future'

X = tiles[['Tile_ID', 'Long2', 'Lat2', 'Long1', 'Lat1', 'Mid_lat', 'Mid_long',
           'Stop_Signs', 'Paving_historical', 'Paving_future', 'Bus_stop',
           'Collisions_Future', 'Collisions_Historical']]
y = tiles[['label']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)

X_train, X_train_lr, y_train, y_train_lr = train_test_split(X_train,
                                                            y_train,
                                                            test_size=0.5)

**RANDOM FOREST**

https://www.dlology.com/blog/simple-guide-on-how-to-generate-roc-plot-for-keras-classifier/

In [ ]:
#def build_model():
#    model = Sequential()
#    model.add(Dense(20, input_dim=8, activation='relu'))
#    model.add(Dense(40, activation='relu'))
#    model.add(Dense(1, activation='sigmoid'))
#    # Compile model
#   model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#    return model

#from keras.wrappers.scikit_learn import KerasClassifier
#keras_model = build_model()
#keras_model.fit(X_train, y_train, epochs=20, batch_size=100, verbose=1)

In [ ]:
#y_pred_keras = keras_model.predict(X_test).ravel()
#fpr_keras, tpr_keras, thresholds_keras = roc_curve(y_test, y_pred_keras)

In [ ]:
#auc_keras = auc(fpr_keras, tpr_keras)

In [ ]:
from sklearn.metrics import accuracy_score
# Supervised transformation based on random forests
rf = RandomForestClassifier(criterion="gini", max_depth=8, n_estimators=30)
rf.fit(X_train, y_train)

y_pred_rf = rf.predict_proba(X_test)[:, 1]
fpr_rf, tpr_rf, thresholds_rf = roc_curve(y_test, y_pred_rf)
auc_rf = auc(fpr_rf, tpr_rf)

<ipython-input-10-4cb46dc52041>:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_train, y_train)


ValueError: ignored

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

#print(accuracy_score(y_test, y_pred_RF))

In [ ]:
plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
#plt.plot(fpr_keras, tpr_keras, label='Keras (area = {:.3f})'.format(auc_keras))
plt.plot(fpr_rf, tpr_rf, label='RF (area = {:.3f})'.format(auc_rf))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve: Random Forest')
plt.legend(loc='best')
plt.show()

**LOGISTIC REGRESSION**

In [ ]:
from sklearn.linear_model import LogisticRegression
# Supervised transformation based on random forests

logr = LogisticRegression(random_state=0, max_iter=1000)
#rf = RandomForestClassifier(max_depth=3, n_estimators=10)
logr.fit(X_train, y_train)

y_pred_logr = logr.predict_proba(X_test)[:, 1]
fpr_logr, tpr_logr, thresholds_logr = roc_curve(y_test, y_pred_logr)
auc_logr = auc(fpr_logr, tpr_logr)

In [ ]:
plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
#plt.plot(fpr_keras, tpr_keras, label='Keras (area = {:.3f})'.format(auc_keras))
plt.plot(fpr_logr, tpr_logr, label='LogR (area = {:.3f})'.format(auc_logr))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve: Logistic Regression')
plt.legend(loc='best')
plt.show()

**KNN Classifier**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

y_pred_knn = knn.predict_proba(X_test)[:, 1]
fpr_knn, tpr_knn, thresholds_knn = roc_curve(y_test, y_pred_knn)
auc_knn = auc(fpr_knn, tpr_knn)

In [ ]:
plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
#plt.plot(fpr_keras, tpr_keras, label='Keras (area = {:.3f})'.format(auc_keras))
plt.plot(fpr_logr, tpr_logr, label='KNN (area = {:.3f})'.format(auc_knn))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve: KNN Classifier')
plt.legend(loc='best')
plt.show()

**Decision Tree**

In [ ]:
from sklearn import tree
dtree = tree.DecisionTreeClassifier()
dtree = dtree.fit(X_train, y_train)

y_pred_dtree = dtree.predict_proba(X_test)[:, 1]
fpr_dtree, tpr_dtree, thresholds_dtree = roc_curve(y_test, y_pred_dtree)
auc_dtree = auc(fpr_dtree, tpr_dtree)

In [ ]:
plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
#plt.plot(fpr_keras, tpr_keras, label='Keras (area = {:.3f})'.format(auc_keras))
plt.plot(fpr_logr, tpr_logr, label='Decision Tree (area = {:.3f})'.format(auc_dtree))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve: Decision Tree')
plt.legend(loc='best')
plt.show()